In [29]:
import sympy as sp
from IPython.display import display, Latex

# Define symbols
x, z, a = sp.symbols('x z a')  # Position and angle
xdot, zdot, adot = sp.symbols('xdot zdot adot')  # Velocities
xddot, zddot, addot = sp.symbols('xddot zddot addot')  # Accelerations
u, w, q = sp.symbols('u w q')  # Optional velocities (if needed)

# Physical parameters
m, g, B, A, rho, Iyy = sp.symbols('m g B A rho I_{yy}')
Cl_alpha, Cd0, Correction, e = sp.symbols('C_{l\\alpha} C_{d0} kappa e') 
Cl_hull, Cd_hull = sp.symbols('C_{l\\hull} C_{d\\hull}')
FT, theta0 = sp.symbols('F_T theta_0')  # Thrust force and angle

# Position-dependent offsets for different reference points
r_xi, r_zi, r_xh, r_zh, r_xt, r_zt, r_xb = sp.symbols(
    'r_{xi} r_{zi} r_{xh} r_{zh} r_{xt} r_{zt} r_{xb}'
)

# Compute common aerodynamic angle of attack
def compute_alpha(r_x, r_z):
    """Calculate the angle of attack α based on r_x and r_z."""
    return sp.atan((zdot - adot * r_x) / (xdot + adot * r_z))

alpha_i = compute_alpha(r_xi, r_zi)  # Angle of attack for 'i' reference
alpha_h = compute_alpha(r_xh, r_zh)  # Angle of attack for 'h' reference


Cl_hull = sp.Rational(2 ,3) * sp.pi
Cd0_hull = 0 # Assuming no hull drag for simplicity

# Define the state vector
X = sp.Matrix([x, z, a, xdot, zdot, adot])
X_dot = sp.Matrix([xdot, zdot, adot, xddot, zddot, addot])

# Reusable term: Aerodynamic drag and lift forces
def aero_force(Cl_alpha, Cd0, r_x, r_z, alpha):
    """Calculate aerodynamic force component based on given offsets and α."""
    velocity_sq = (xdot + adot * r_z)**2 + (zdot - adot * r_x)**2
    lift = Cl_alpha * Correction * alpha
    drag = Cd0 + (lift / (sp.pi * A * e))**2  # Lift-induced drag correction
    return 0.5 * A * rho * velocity_sq * drag, 0.5 * A * rho * velocity_sq * lift

F_drag_i, F_lift_i = aero_force(Cl_alpha, Cd0, r_xi, r_zi, alpha_i)
F_drag_h, F_lift_h = aero_force(Cl_hull, Cd0_hull, r_xh, r_zh, alpha_h)

# Equations of Motion
f1 = xdot  # dx/dt = xdot
f2 = zdot  # dz/dt = zdot
f3 = adot  # da/dt = adot

# ẍ equation
f4 = (
    (1 / m) * (
        -(m * g - B) * sp.sin(a) 
        + FT * sp.cos(theta0 - a)
        - F_drag_i 
        - F_drag_h
        )
        - adot * zdot
)

# z̈ equation
f5 = (
    (1 / m) * (
        (m * g - B) * sp.cos(a) 
        + FT * sp.sin(theta0 - a)
        + F_lift_i
        + F_lift_h
       )
        + adot * xdot
)

# ä equation
f6 = (1 / Iyy) * (
    B * r_xb * sp.cos(a) +
    FT * (-sp.cos(theta0 - a) * r_zt + sp.sin(theta0 - a) * r_xt) +
    F_lift_i * (r_xi) +
    F_drag_i * (-r_zi) +
    F_lift_h * (r_xh) +
    F_drag_h * (-r_zh)
    )


# Form a dictionary of the associated equations
equations = {
    xdot  : f1,
    zdot  : f2,
    adot  : f3,
    xddot : f4,
    zddot : f5,
    addot : f6,
}

F = sp.Matrix([f1,f2, f3, f4, f5, f6])


## Evaluated Functions:

In [32]:
# Display the equations as is (no simplification)
for variable, eq in equations.items():
    display(Latex(f"${sp.latex(variable)} = {sp.latex(eq)}$"))

# Display the equations with simplification
# for i, eq in enumerate(X_dot):
#     # Simplify or force fraction representation where necessary
#     simplified_eq = sp.simplify(eq)
#     display(Latex(f"${sp.latex(X[i])} = {sp.latex(simplified_eq, mode='equation')}$"))



<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [33]:
# Compute the Jacobian matrix
Jacobian = F.jacobian(X)

# Display partial derivatives
for i in range(Jacobian.shape[0]):
    for j in range(Jacobian.shape[1]):
        partial = sp.simplify(Jacobian[i, j])
        disp_partial = f"\\frac{{\\partial f_{i+1}}}{{\\partial {sp.latex(X_dot[j])}}} = {sp.latex(partial)}\n"
        display(Latex("$" + disp_partial + "$"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [40]:
display(Latex(f"${sp.latex(Jacobian)}$"))

<IPython.core.display.Latex object>

In [42]:
display(Latex(f'${sp.latex(F)}$'))

<IPython.core.display.Latex object>